In [1]:
import pandas as pd
import string
import nltk
from autocorrect import Speller
import re

## Get daily dialog data

In [2]:
# Get daily dialog data

with open('datasets/data_raw/dailydialog/dialogues_text.txt', encoding= 'utf-8') as f:
    dialogue = f.readlines()

with open('datasets/data_raw/dailydialog/dialogues_emotion.txt', encoding= 'utf-8') as f:
    emotion = f.readlines()

d=[]
sentences = []
for line in dialogue:
    line = line.replace('\n', '')
    d.append(line.split('__eou__'))

for lis in d:
    for sentence in lis:
        sentences.append(sentence)

sentences = list(filter(None,sentences))

emotion = ' '.join(emotion).split()

df = pd.DataFrame(zip(emotion, sentences), columns=['emotion','text'])
df.to_csv('datasets/dailydialog.csv')
df.head()


,emotion,text
0,2,The kitchen stinks .
1,0,I'll throw out the garbage .
2,4,"So Dick , how about getting some coffee for to..."
3,2,Coffee ? I don ’ t honestly like that kind of...
4,0,"Come on , you can at least try a little , bes..."


## Get emotion stimulus data

In [3]:
# Get emotion stimulus data

with open('datasets/data_raw/emotion_stimulus/Emotion Cause.txt', encoding= 'utf-8') as f:
    emotion_stimulus = f.readlines()

with open('datasets/data_raw/emotion_stimulus/No Cause.txt', encoding= 'utf-8') as f:
    emotion_stimulus_no_cause = f.readlines()

emotion_stimulus_all = emotion_stimulus+emotion_stimulus_no_cause
print(len(emotion_stimulus_all))

2414


In [4]:
df_emotion_stimulus = pd.DataFrame(emotion_stimulus_all,columns=['text'])

df_emotion_stimulus['emotion'] = df_emotion_stimulus['text'].apply(lambda x: str(x.split('>')[0]))

In [5]:
df_emotion_stimulus.head()

,text,emotion
0,<happy>I suppose I am happy <cause>being so ` ...,<happy
1,<happy>Lennox has always truly wanted to fight...,<happy
2,"<happy>He was a professional musician now , st...",<happy
3,<happy>Holmes is happy <cause>having the freed...,<happy
4,<happy>I had problems with tutors trying to en...,<happy


In [6]:
def clean_text(text):
    import re
    rep = {'<cause>': '',
           r'<\\cause>': '',
           '<happy>': '',
           '<sad>': '',
           '<anger>': '',
           '<fear>': '',
           '<suprise>': '',
           '<shame>': '',
           '<disgust>': ''}

    for item in rep.keys():
        text = re.sub(item, rep[item], text)
        
    return text


df_emotion_stimulus['text'] =  df_emotion_stimulus['text'].apply(lambda x: clean_text(x))

df_emotion_stimulus['emotion'] = df_emotion_stimulus['emotion'].apply(lambda x: x.replace('<',''))
df_emotion_stimulus.to_csv('datasets/emotion_stimulus.csv')
df_emotion_stimulus= df_emotion_stimulus[['emotion','text']]

df_emotion_stimulus.head()

,emotion,text
0,happy,I suppose I am happy being so ` tiny' ; it mea...
1,happy,Lennox has always truly wanted to fight for th...
2,happy,"He was a professional musician now , still sen..."
3,happy,Holmes is happy having the freedom of the hous...
4,happy,I had problems with tutors trying to encourage...


## Get Isear data

In [7]:
df_isear = pd.read_csv('datasets/data_raw/isear.csv', sep='|',header=None, names=['ID','CITY','COUN','SUBJ','SEX','AGE','RELI','PRAC','FOCC','MOCC','FIEL','EMOT','WHEN','LONG','INTS','ERGO','TROPHO','TEMPER','EXPRES','MOVE','EXP1','EXP2','EXP10','PARAL','CON','EXPC','PLEA','PLAN','FAIR','CAUS','COPING','MORL','SELF','RELA','VERBAL','NEUTRO','Field1','Field3','Field2','MYKEY','SIT','STATE'])

df_isear.columns = ['CITY','COUN','SUBJ','SEX','AGE','RELI','PRAC','FOCC','MOCC','FIEL','EMOT','WHEN','LONG','INTS','ERGO','TROPHO','TEMPER','EXPRES','MOVE','EXP1','EXP2','EXP10','PARAL','CON','EXPC','PLEA','PLAN','FAIR','CAUS','COPING','MORL','SELF','RELA','VERBAL','NEUTRO','Field1','Field3','Field2','MYKEY','SIT','STATE','']

df_isear = df_isear[['Field1','SIT']]
df_isear = df_isear[df_isear.Field1!='Field1']
df_isear.reset_index(drop=True, inplace=True)
df_isear.columns=['emotion','text']
df_isear.to_csv('datasets/isear.csv')
df_isear.head()

,emotion,text
0,joy,"During the period of falling in love, each tim..."
1,fear,When I was involved in a traffic accident.
2,anger,When I was driving home after several days of...
3,sadness,When I lost the person who meant the most to me.
4,disgust,The time I knocked a deer down - the sight of ...


In [8]:
frames = [df_isear, df_emotion_stimulus,df]
df = pd.concat(frames)

In [9]:
df['emotion'].value_counts()

0           85572
4           12885
6            1823
anger        1579
fear         1518
shame        1242
disgust      1191
5            1150
sadness      1096
joy          1094
guilt        1093
1            1022
sad           575
happy         479
2             353
surprise      213
3             174
Name: emotion, dtype: int64

In [10]:

def clean_text(text):
    import re
    replace_emotions = {'0': 'neutral',
                    '1': 'anger',
                    '2': 'disgust',
                    '3': 'fear',
                    '4': 'happy',
                    '5': 'sad',
                    '6': 'suprise',
                    'surprise': 'suprise',
                    'sadness': 'sad'}

    for item in replace_emotions.keys():
        text = re.sub(item, replace_emotions[item], text)
        
    return text

df['emotion'] =  df['emotion'].apply(lambda x: clean_text(x))
df.to_csv('datasets/dataset_complete.csv',index=False )

In [11]:
pd.set_option('display.max_colwidth',0)
data = pd.read_csv('datasets/dataset_complete.csv',encoding='utf-8')
data.head()

,emotion,text
0,joy,"During the period of falling in love, each time that we met and á especially when we had not met for a long time."
1,fear,When I was involved in a traffic accident.
2,anger,"When I was driving home after several days of hard work, there á was a motorist ahead of me who was driving at 50 km/hour and á refused, despite his low speeed to let me overtake."
3,sad,When I lost the person who meant the most to me.
4,disgust,"The time I knocked a deer down - the sight of the animal's á injuries and helplessness. The realization that the animal was á so badly hurt that it had to be put down, and when the animal á screamed at the moment of death."


In [12]:

data['emotion'].value_counts()

neutral    85572
happy      13364
sad        2821 
anger      2601 
suprise    2036 
fear       1692 
disgust    1544 
shame      1242 
joy        1094 
guilt      1093 
Name: emotion, dtype: int64

In [13]:
data = data.sort_values('emotion').reset_index(drop=True)

data[data.emotion=='happy']

,emotion,text
6930,happy,"I ’ m thinking of making an album , you know ."
6931,happy,"I ’ Ve gotta hand it to you , you really did a good job !"
6932,happy,"Pretty good . Actually , I'm quite busy with work . By the way , say ' hello ' to your husband ."
6933,happy,Great !
6934,happy,When and where ?
...,...,...
20289,happy,I see . Anything else ?
20290,happy,Do you ?
20291,happy,Thank you . I ’ ll try my best .
20292,happy,Really ? I am glad to meet you .


In [14]:
data=data.drop(data.index[6930:17700],axis=0).reset_index(drop=True)
data['emotion'].value_counts()

neutral    85572
sad        2821 
anger      2601 
happy      2594 
suprise    2036 
fear       1692 
disgust    1544 
shame      1242 
joy        1094 
guilt      1093 
Name: emotion, dtype: int64

In [15]:
data[data.emotion=='neutral']

,emotion,text
10618,neutral,That wouldn't seem right .
10619,neutral,"That's right , it really would be ."
10620,neutral,I would rather deal with the winter than the summer .
10621,neutral,"Sure . They are my kids , Alex and Kathy Stern ."
10622,neutral,I will see you tomorrow .
...,...,...
96185,neutral,I didn ’ t realize it was a romance novel . What ’ s the main storyline ?
96186,neutral,It ’ s basically about a father who tried to marry off one of his five girls .
96187,neutral,I ’ m not surprised . That article certainly added fuel to the debate . Ok . Thanks . Can you leave two articles and the advertisements with me ? I ’ d like to read them .
96188,neutral,That's really exciting . How many months are you pregnant ?


In [16]:
data = data.drop(data.index[10618:93800], axis=0).reset_index(drop=True)

data['emotion'].value_counts()

sad        2821
anger      2601
happy      2594
neutral    2390
suprise    2036
fear       1692
disgust    1544
shame      1242
joy        1094
guilt      1093
Name: emotion, dtype: int64

In [17]:

stopword = nltk.corpus.stopwords.words('english')

def clean_text(text):

    # Replace or remove the characters 
    replacement_dict={"’":"'", # Replace ’ with '
                      "'s":" is", # Replace the contractions
                      "n't": " not",
                      "'m": " am",
                      "'ll": " will",
                      "'d": " would",
                      "'ve": " have",
                      "'re": " are",
                      "\W+": " "
                      } 

    for item in replacement_dict.keys():
        text=re.sub(item,replacement_dict[item], text)

    text = ''.join([char for char in text if char not in string.punctuation])

    text=text.lower()

    return text

data= data.applymap(lambda x: clean_text(x))

data.to_csv('datasets/dataset_complete_balanced.csv', index=False)


